# Tabulation of Fractional Cover data within shapefile polygons

**What does this notebook do?**

This notebook is a pilot collaboration between Geoscience Australia and Australian Bureau of Statistics. The purpose of the notebook is to use a shapefile of SA2 boundaries to load Fractional Cover (FC) dataset, complete zonal statistics for each SA2 boundary, and  then tabulate monthly and annual results. Tabulated results show the percentage of the SA2 area that is classified as green vegetation (Photosythetic vegetation (PV)), brown vegetation (Non-Photosythetic vegetation (NPV)), bare soil (BS), water (open surface water), or unclassified (cloud or other). 

**Requirements**

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles`

`module load dea/20181213`


**Background**

Data from the Landsat 5,7 and 8 satellite missions are accessible through Digital Earth Australia (DEA). The code snippets in this notebook will let you retrieve tabulate the FC data stored in DEA.


**How to use this notebook**

A basic understanding of any programming language is desirable but one doesn't have to be an expert Python programmer to manipulate the code to get and display the data.This doc applies to the following Landsat satellites, Fractional Cover bands and the WOfS dataset:

- Landsat 5
- Landsat 7
- Landsat 8
- PV - Photosythetic vegetation
- NPV - Non-Photosythetic vegetation
- BS - Bare Soil
- UE - Unmixing Error
- Water Observations from Space (WOFs)
- WOfS Feature Layer (WOFL)

**Bugs still to fix**

- NA


**Errors or bugs**

If you find an error or bug in this notebook, please contact erin.telfer@ga.gov.au.


## 1. Import Libraries

In [ ]:
%matplotlib inline

from datetime import time, datetime
import os.path

from matplotlib import pyplot as plt
import pandas as pd
import numpy
import csv
import xarray as xr
import rasterio.features
import fiona
import dask
from dask.delayed import delayed
from dask.distributed import LocalCluster, Client
import tempfile
from datetime import *

import datacube
from datacube import Datacube
from datacube.virtual import construct, construct_from_yaml
from datacube.utils.geometry import CRS

## 2. USER INPUT

In [ ]:
#Do you want to create stackplots for each SA2? If yes, enter "True". If no, enter "False"
create_stackplots = "False"

#What years are you interested in?
start_year = 2015
end_year = 2017

#Enter input batch details to help track each run of the code
initials = 'ET' #initial of the person running the code
batch_sa2_size = '0_to_10km2' #if you have split the SA2 into smaller subsets of data by size, add size range
run_date = '16Aug19' #the date today
run_notes = 'Rerun of SA2 between 0 to 10 km2 for the date range 2015 to 2017' #a brief note to describe the run

## 3. Set directory and shapefile details

In [ ]:
#Set folder location and shapefile name
shapefile_path = '/g/data/r78/ext547/abs/input/SA2_2016_AUST.shp'
#Set output folder location
output_path = '/g/data/r78/ext547/abs/output/'
#Save variable name for output csv
name_of_output_file =f'tabulate_FC_{run_date}_{batch_sa2_size}_{start_year}_{end_year}'

In [ ]:
#Years of interest are saved in expected format
time_range = (str(start_year), str(end_year))
print(time_range)

## 4. Set up a local dask cluster
Some calculations take more memory than is available on a system.  By breaking the data up into chunks, we can chain a sequence of operations together, and work on the data a small piece at a time.

This lets several processes work at the same time, and manage total memory usage for the calculations.

In more advanced setups, we can distribute the work across multiple computers, using all of their memory and CPU power.

* We set `n_workers` to be the number of worker applications we want to run in the background, doing the processing of the chunk-based steps we have chained together.
* The `mem_per_worker` setting defines how much memory at most is available to each of the workers.
* `chunk_size` sets the width and height of the chunk in pixels of the size will break up the data into.

VDI has 8 CPUs available, and a total of 32GB of memory, but you will typically be sharing those with several (2-10) other users.   

In [ ]:
#Set up dask cluster
n_workers = 7
threads_per_worker=1
mem_per_worker = 8e9  # 8e9 is 8GB (8,000,000,000 bytes)

chunk_size = {'time': 1, 'x': 2000, 'y': 2000}

In [ ]:
cluster = LocalCluster(local_dir=tempfile.gettempdir(), 
                       n_workers=n_workers, 
                       threads_per_worker=threads_per_worker,
                       memory_limit=mem_per_worker)
client = Client(cluster)
dask.config.set(get=client.get)
by_slice=True
client

We also get a dashboard to see how the system is running, by clicking the link above after the cell has been run.

## 5. Connect to the Datacube 

In [ ]:
dc = Datacube()

## 6. Construct the virtual product

In [ ]:
#Remove Landsat 7 scenes with the Scan Line Correction (SLC) missing data
LS7_BROKEN_DATE = datetime(2003, 5, 31)
is_pre_slc_failure = lambda dataset: dataset.center_time < LS7_BROKEN_DATE

In [ ]:
#Create function to ensure wofls in correct format
def wofls_fuser(dest, src):
    where_nodata = (src & 1) == 0
    numpy.copyto(dest, src, where=where_nodata)
    return dest

In [ ]:
#Create virtual product so that datacube data can be loaded effectively within memory
fc_and_water_yaml = """
        juxtapose:
          - collate:
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls5_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                      - transform: make_mask
                        input:
                            product: ls5_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls7_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                        # dataset_predicate: __main__.is_pre_slc_failure
                      - transform: make_mask
                        input:
                            product: ls7_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls8_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                      - transform: make_mask
                        input:
                            product: ls8_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
          - transform: make_mask
            input:
                product: wofs_albers
                group_by: solar_day
                fuse_func: __main__.wofls_fuser
            flags:
                wet: true
            mask_measurement_name: water
"""
fc_and_water = construct_from_yaml(fc_and_water_yaml)

## 7. Set up functions

In [ ]:
def geometry_mask(geoms, geobox, all_touched=False, invert=False, chunks=None):
    """
    Create a mask from shapes.

    By default, mask is intended for use as a
    numpy mask, where pixels that overlap shapes are False.
    :param list[Geometry] geoms: geometries to be rasterized
    :param datacube.utils.GeoBox geobox:
    :param bool all_touched: If True, all pixels touched by geometries will be burned in. If
                             false, only pixels whose center is within the polygon or that
                             are selected by Bresenham's line algorithm will be burned in.
    :param bool invert: If True, mask will be True for pixels that overlap shapes.
    """
    data = rasterio.features.geometry_mask([geom.to_crs(geobox.crs) for geom in geoms],
                                           out_shape=geobox.shape,
                                           transform=geobox.affine,
                                           all_touched=all_touched,
                                           invert=invert)
    if chunks is not None:
        data = dask.array.from_array(data, chunks=tuple(chunks[d] for d in geobox.dims))
        
    coords = [xr.DataArray(data=coord.values, name=dim, dims=[dim], attrs={'units': coord.units}) 
              for dim, coord in geobox.coords.items()]
    return xr.DataArray(data, coords=coords)

In [ ]:
def get_shapes(shape_file):
    """
    Extract spatial inforamtion from polygons within shapefile
    """
    with fiona.open(shape_file) as shapes:
        crs = datacube.utils.geometry.CRS(shapes.crs_wkt)
        for shape in shapes:
            if shape['geometry'] is None:
                continue
            geom = datacube.utils.geometry.Geometry(shape['geometry'], crs=crs)
            geom = geom.to_crs(CRS('EPSG:3577'))
            yield geom, shape['properties']

In [ ]:
def fc_and_water_summary(data, mask_int):
    """
    Calculate the percentage and the area of each FC component, 
    water, and unclassified data within each SA2 boundary. 
    """
    # Where there is water, set the FC bands to 0%
    valid_data = numpy.isfinite(data['BS'])
    fc = data[['PV', 'NPV', 'BS']].where(data.water==0, other=0)
    fc['water'] = data.water.where(valid_data) * numpy.float32(100)
    fc = fc.apply(lambda data_array: data_array.clip(0, 100).where(valid_data))
    has_data = valid_data.groupby(valid_data.time.astype('datetime64[M]'), squeeze=False).sum(dim='time', skipna=False)
    has_data = has_data.sum(dim=['x','y'], skipna=True) / (mask_int / 100)
    
    # Flatten to a monthly mean
    fc = fc.groupby(fc.time.astype('datetime64[M]'), squeeze=False).mean(dim='time', skipna=True)
    
    # Calculate percentage of cover based on area of mask
    percentage = fc.sum(dim=['x','y']) * (100 / mask_int)
    for da in percentage.data_vars.values():
        da.attrs['units'] = '%'
    fc_sum = fc.sum(dim=['x','y'])
    fc_sum = fc_sum.PV + fc_sum.NPV + fc_sum.BS + fc_sum.water
    
    #Calculate the amount of SA2 that is unclassified (e.g. not FC or water)
    unclass_size = mask_int - fc_sum
    fc['unclassified'] = unclass_size     
    unclass_percentage = unclass_size * (100 / mask_int)
    percentage['unclassified'] = unclass_percentage
    
    #Calculate area of fc components   
    pixel_area_in_metres2 = 25 * 25
    m2_to_km2 = (1 / 1_000_000)
    percent_to_fraction = (1 / 100)
    area = (fc * (pixel_area_in_metres2 * m2_to_km2 * percent_to_fraction)).sum(dim=['x','y'])
    area = area.rename({'BS': 'BS_area', 
                        'PV': 'PV_area', 
                        'NPV': 'NPV_area', 
                        'water': 'water_area',
                        'unclassified':'unclassified_area'})
    for da in area.data_vars.values():
        da.attrs['units'] = 'km^2'
    fc = percentage.merge(area)   
    fc['average_data_count'] = has_data
    
    return fc

In [ ]:
def plot_stacked(ds, sa2_id,plot_title='title', show=True):
    """
    Create and save a stacked plot to visualise FC components
    """
    if not show:
        plt.ioff()
        
    fig,ax = plt.subplots(figsize=(10,5))
    ax.stackplot(ds.dropna(dim='time').time.data, 
                 ds.dropna(dim='time').PV,
                 ds.dropna(dim='time').NPV, 
                 ds.dropna(dim='time').BS, 
                 ds.dropna(dim='time').water, 
                 ds.dropna(dim='time').unclassified, 
                 colors = ['darkolivegreen','olive','tan','darkblue','red'], 
                 labels=['Green veg','Brown veg','Bare soil','Water','Unclassified',])
    plt.legend(loc='lower center', ncol = 5)
    plt.title(f'FC components: SA2 ID {sa2_id}', size=12)
    plt.ylabel('Percentage (%)', size=12) #Set Y label
    plt.xlabel('Date', size=12) #Set X label
    
    plt.savefig(f'{output_path}/{sa2_id}_{plot_title}.png');
    plt.close(fig)
    
    # Turn interactive back on
    if not show:
        plt.show()

In [ ]:
def month_splitter(start_year, end_year_inclusive):
    """ 
    Split specified years into months 
    """
    yield from (str(p) for p in pd.period_range(start=pd.Period(start_year).start_time, 
                               end=pd.Period(end_year_inclusive).end_time, 
                               freq='M'))

In [ ]:
def output_csv(input_ds, sa2_id, sa2_name, sa2_size, monthly_or_annual='frequency'):
    """
    Save tabulated data into a csv
    """
    input_ds = input_ds.to_dataframe()
    input_ds.insert(0,'SA2_id', sa2_id)
    input_ds.insert(1,'SA2_name', sa2_name)
    input_ds.insert(2,'SA2_size', sa2_size)
    input_ds.to_csv(f"{output_path}/{name_of_output_file}_{monthly_or_annual}.csv",mode='a',header=False)  

In [ ]:
#create FC tabulation function
monthly_values = None

def process_area(geometry, sa2_id, sa2_name, sa2_size, time_range):
    """Process the values"""
    monthly_values = []
    annual_values = []
    mask = None
    
    # Virtual Products have a bug handling geometry objects, use this instead of `geopolygon=geometry` for now
    search_terms = {
        'x': (geometry.envelope.left, geometry.envelope.right),
        'y': (geometry.envelope.top, geometry.envelope.bottom),
        'crs': str(geometry.crs),
    }  

    for sub_time_range in month_splitter(time_range[0], time_range[-1]):
        print(sub_time_range)
        try:
            data = fc_and_water.load(dc, dask_chunks=chunk_size, 
                                     time=sub_time_range, 
                                     **search_terms)
        except ValueError:
            print(f'    No data for {sub_time_range} , skipping...')
            continue

        if mask is None:
            mask = geometry_mask([geometry], data.geobox, invert=True, chunks=data.chunks)
            mask_int = mask * 1
            mask_int = mask_int.sum() * 100
            mask_int.load()
        # Mask data
        data = data.where(mask)
        monthly_data = fc_and_water_summary(data, mask_int)
        monthly_data = client.compute(monthly_data, sync=by_slice)
        monthly_values.append(monthly_data)
        
    monthly_values = xr.concat(monthly_values, dim='time')
    monthly_qa = xr.Dataset(coords={'time': monthly_values.time})
    monthly_qa['has_values'] = (('time',), [1] * monthly_values.time.size)
    monthly_qa['has_10'] = monthly_values['unclassified'] <= 10
    monthly_values = monthly_values.where(monthly_values['unclassified'] < 10).dropna(dim='time')
    annual_values = monthly_values.resample(time='1YS').mean(dim='time', skipna=True)
    
    # Replace the mean of the monthly obs count with the annual total obs count
    annual_avg_data_count = monthly_values['average_data_count'].resample(time='1YS').sum(dim='time', skipna=True)
    annual_values['average_data_count'] = annual_avg_data_count
    annual_qa = monthly_qa.resample(time='1YS').sum(dim='time', skipna=True)
    annual_qa = 12 - annual_qa
    annual_values = annual_values.merge(annual_qa)
    annual_values['time'] = annual_values.time.dt.year

    print(f"Calculation complete for annual values")
    
    if create_stackplots == "True":
        plot_stacked(monthly_values, sa2_id, plot_title='monthly_plot_wofs',show=False)
        plot_stacked(annual_values, sa2_id, plot_title='annual_plot_wofs', show=False)
    else:
        print(f"Do not make stackplot")
    
    #append  outputs to csv
    print("All data loaded, save to csv")
    output_csv(monthly_values, sa2_id, sa2_name, sa2_size, monthly_or_annual='monthly')
    output_csv(annual_values, sa2_id, sa2_name, sa2_size, monthly_or_annual='annual')
    
    return annual_values, monthly_values
    
    print(f"SA2 {sa2_id} done")

## 8. Set up the query
For each year and polygon query the product, apply the geometry mask and compute the fractional cover stats

Using `client.compute()` lets us use the monthly results in calculating the annual results at the same time.

In [ ]:
#The following code does not need to be edited.

#Obtain spatial information from shapefile
shape_file = os.path.expanduser(f'{shapefile_path}')
shapes = list(get_shapes(shape_file))

#A list of sa2 that do not have a spatial footprint (e.g. "Migratory - Offshore - Shipping" OR "No usual address") 
#or SA2 that are outside of the DEA coverage, e.g. Christmas Island or Norfolk Island. 
sa2_outside_DEA = ['901011001', '108031161', '901021002', '901041004', 
                   '197979799', '199999499', '297979799', '299999499', 
                   '397979799', '399999499', '497979799', '499999499', 
                   '597979799', '599999499', '697979799', '699999499', 
                   '797979799', '799999499', '897979799', '899999499', 
                   '997979799', '999999499']

#remove the SA2 that are outside of the coverage of DEA FC dataset
for sa2_outside in sa2_outside_DEA:
    shapes = [(g,p) for g, p in shapes if str(p['SA2_MAIN16']) != sa2_outside]
    
print(len(shapes))

In [ ]:
#The follow code allows the use to choose one specific SA2 base. Hash/unhash the specific line of code.
#The user can filter by things such as SA2 ID, or SA2 size.

# shapes = [(g,p) for g, p in shapes if str(p['SA2_MAIN16']) == '801041120'] #choose based on SA2 ID
# shapes = [(g,p) for g, p in shapes if int(p['AREASQKM16']) <= 0.99]  #choose based on SA2 size, smaller than value
shapes = [(g,p) for g, p in shapes if 0 < int(p['AREASQKM16']) <= 20] #choose based on SA2 size, between two value

print(len(shapes))

## 9. Create empty CSVs to save results

In [ ]:
#Create empty CSV with specified headings
header = ['DATE','SA2_MAIN16', 'SA2_NAME16', 'AREASQKM16','PV_PERCENTAGE','NPV_PERCENTAGE','BS_PERCENTAGE'
          ,'WOFL_PERCENTAGE', 'UNCLASSIFIED_PERCENTAGE','PV_AREA_SQKM_ALBERS3577','NPV_AREA_SQKM_ALBERS3577',
          'BS_AREA_SQKM_ALBERS3577','WOFL_AREA_SQKM_ALBERS3577','UNCLASSIFIED_AREA_SQKM_ALBERS3577', 
          'AVERAGE_OBSERVATION_COUNT']

additional_annual_header = ['MONTHS_WITH_NO_DATA', 'MONTHS_WITH_LOW_DATA']

error_header = ['','RUN_NAME','SA2_MAIN16','DATE_RANGE','ERROR_MESSAGE']

with open(f"{output_path}/{name_of_output_file}_annual.csv","w") as outcsv: #create csv to save output and add header text
    writer = csv.writer(outcsv)
    writer.writerow(header + additional_annual_header)
    
with open(f"{output_path}/{name_of_output_file}_monthly.csv","w") as outcsv: #create csv to save output and add header text
    writer = csv.writer(outcsv)
    writer.writerow(header)
    
with open(f"{output_path}/{name_of_output_file}_error_log.csv","w") as outcsv: #create csv to save output
    writer = csv.writer(outcsv)
    writer.writerow(error_header)    

## 10. Run tabulation script

In [ ]:
#Run the tabulation process while saving metadata for the run.
#The script loops through each SA2 and tabulates the FC results, saving to csv.
#additionally the script has been set up to save run metadata and error information (if any)

#save batch/run metadata
run_start = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
count_of_sa2_complete = 0
count_of_sa2_error = 0
count_of_sa2_all = 0
sa2_min_size = 500_0000
sa2_max_size = 0

#save batch information out to csv
batch_start_information = pd.DataFrame([[name_of_output_file],[run_notes],[initials],[run_start]]).T
batch_start_information.to_csv(f"{output_path}/tabulate_FC_run_log.csv",mode='a',header=False)

#loop through SA2 and use tabulation script
for geometry, properties in shapes:
    count_of_sa2_all += 1 #metadata
    
    #print SA2 information to screen
    print(count_of_sa2_all)
    sa2_id = str(properties['SA2_MAIN16'])
    sa2_name = str(properties['SA2_NAME16'])
    sa2_size = str(properties['AREASQKM16'])
    print(f"SA2 ID: {sa2_id}, size: {sa2_size}km^2, time: {time_range}")
    
    #loop through SA2 size and save the smallest and largest SA2 values
    if float(sa2_min_size) > float(sa2_size):
        sa2_min_size = sa2_size #metadata
    if float(sa2_max_size) < float(sa2_size):
        sa2_max_size = sa2_size #metadata
    
    #loop through months/years and tabulate FC results
    try:
        annual_vals, monthly_vals = process_area(geometry, sa2_id, sa2_name, sa2_size, time_range)
        count_of_sa2_complete += 1
    #if there are errors, save information to csv to record the name of SA2 and type of error
    except Exception as e:
            print(f"Could not process {sa2_id}: {e}")
            error_information = pd.DataFrame([[name_of_output_file],[sa2_id],[time_range], [e]]).T
            error_information.to_csv(f"{output_path}/{name_of_output_file}_error_log.csv",mode='a',header=False)
            count_of_sa2_error += 1
            client.restart()

#save batch/run information
run_end = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #metadata
run_seconds = (datetime.strptime(run_end,'%Y-%m-%d %H:%M:%S') - datetime.strptime(run_start,'%Y-%m-%d %H:%M:%S')).total_seconds() #metadata

#save run/batch metadata information to csv
batch_information = pd.DataFrame([[name_of_output_file],[run_notes],[initials],
                                  [run_start],[run_end],[run_seconds],[sa2_max_size],
                                  [sa2_min_size],[time_range],[count_of_sa2_all],
                                  [count_of_sa2_complete], [count_of_sa2_error],]).T
batch_information.to_csv(f"{output_path}/tabulate_FC_run_log.csv",mode='a',header=False)

In [ ]:
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")